In [5]:
import pandas as pd
from profiler import profiler
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

# CREATE DATAFRAME
df = pd.read_excel("Megastore Dataset.xlsx")

# NO DUPLICATE ROWS FOUND
# df = df.drop_duplicates()

c:\Users\ajmar\miniconda3\Lib\site-packages\openpyxl\packaging\core.py:99: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow()
c:\Users\ajmar\miniconda3\Lib\site-packages\openpyxl\packaging\core.py:99: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow()


In [8]:
# DEFINE ENCODED VARIABLES 
ordinal_variables = ['Order Priority', 'Segment']
nominal_variables = ['Category', 'Region']

# ORDINAL ENCODING
ordinal_encoder = OrdinalEncoder(categories=[['Low','Medium','High','Critical'],['Consumer','Home Office','Corporate']])
df[ordinal_variables] = ordinal_encoder.fit_transform(df[ordinal_variables])

# ONE-HOT ENCODING
df = pd.get_dummies(df, columns=nominal_variables)



In [11]:
#frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True)
print(df.head())

   Row ID                  Order ID Order Date  Ship Date       Ship Mode  \
0   33712  CA-2013-PP18955140-41461 2013-07-06 2013-07-11  Standard Class   
1   33326  CA-2013-GP14740140-41604 2013-11-26 2013-11-30  Standard Class   
2   33327  CA-2013-GP14740140-41604 2013-11-26 2013-11-30  Standard Class   
3   33325  CA-2013-GP14740140-41604 2013-11-26 2013-11-30  Standard Class   
4   33328  CA-2013-GP14740140-41604 2013-11-26 2013-11-30  Standard Class   

    Customer ID Customer Name  Segment  Postal Code        City  ...  \
0  PP-189551406    Paul Prost      1.0       1040.0     Holyoke  ...   
1  GP-147401406   Guy Phonely      2.0       1453.0  Leominster  ...   
2  GP-147401406   Guy Phonely      2.0       1453.0  Leominster  ...   
3  GP-147401406   Guy Phonely      2.0       1453.0  Leominster  ...   
4  GP-147401406   Guy Phonely      2.0       1453.0  Leominster  ...   

  Region_South America Region_Southeastern Asia Region_Southern Africa  \
0                False        